# File Name: simple_pretraining_builder.ipynb
### Location: Chapter 9
### Purpose: 
#####             1. Need to fill up 
##### Dependency: simple-sageMaker-bedrock.ipynb at Chapter 3 should work properly. 
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [3]:
%%time 

%pip install --upgrade pip
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "langchain" \
    "typing_extensions" \
    "langchain-community" \
    "pypdf" \
    "urllib3" \
    "jsonlines" \
    "datasets" \
    "pandas" \
    "matplotlib" \
    "ipywidgets>=7,<8"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely 

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [13]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [14]:
import warnings
import json
import os
import sys
import boto3 
import time
import pprint
from datasets import load_dataset
import random
import jsonlines
import botocore
from datetime import datetime
from botocore.exceptions import ClientError
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

### Ignore warning 

In [15]:
warnings.filterwarnings('ignore')

## Define important environment variable

In [16]:
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    boto3_session_name = boto3.session.Session()

    # Retrieve the current AWS region from the session
    aws_region_name = boto3_session_name.region_name
    
    # Create a new Boto3 bedrock client to interact with AWS services
    boto3_bedrock_client = boto3.client('bedrock')
    
    # Create a new Boto3 bedrock runtime client to interact with AWS services
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime')
    
    # Create an STS client to interact with AWS Security Token Service (STS)
    sts_client = boto3.client('sts')
    
    # Create an STS client to interact with AWS Security Token Service (STS)
    iam_client = boto3.client('iam')
    
    # Create an S3 client to interact with Amazon S3
    s3_client = boto3.client('s3')

    # Get the AWS account ID of the caller
    aws_account_id = sts_client.get_caller_identity()["Account"]
    
    # Generate a random suffix number between 200 and 900
    random_suffix = random.randrange(200, 900)
    
    # Generate a suffix using the region and account ID for the S3 bucket name
    s3_suffix = f"{aws_region_name}-{aws_account_id}"

    # Define the name of the S3 bucket (you can replace this with your actual bucket name)
    s3_bucket_name = f'bedrock-kb-{s3_suffix}-{random_suffix}'
    
    # Store all variables in a dictionary
    variables_store = {
        "boto3_session_name": boto3_session_name,
        "aws_region_name": aws_region_name,
        "boto3_bedrock_client": boto3_bedrock_client,
        "random_suffix": random_suffix,
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client,
        "s3_suffix": s3_suffix,
        "s3_bucket_name": s3_bucket_name,
        "sts_client": sts_client,
        "aws_account_id": aws_account_id,
        "iam_client":iam_client,
        "s3_client": s3_client
    }

    # Print all variables
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


boto3_session_name: Session(region_name='us-east-1')
aws_region_name: us-east-1
boto3_bedrock_client: <botocore.client.Bedrock object at 0x7efe513119d0>
random_suffix: 565
boto3_bedrock_runtime_client: <botocore.client.BedrockRuntime object at 0x7efe5135b690>
s3_suffix: us-east-1-027437746815
s3_bucket_name: bedrock-kb-us-east-1-027437746815-565
sts_client: <botocore.client.STS object at 0x7efe51380bd0>
aws_account_id: 027437746815
iam_client: <botocore.client.IAM object at 0x7efe2ef44390>
s3_client: <botocore.client.S3 object at 0x7efe2ef7d9d0>


### ---------------
##### The provided code snippet uses the AWS Boto3 library to manage an Amazon S3 bucket for a knowledge base data source. It begins by creating an S3 client and defines a bucket name, s3_bucket_name. A function, check_bucket_exists, checks whether the specified bucket exists by attempting to retrieve its metadata using the head_bucket method. If the bucket exists, a message is printed confirming its existence. If it does not exist (error code '404'), the function returns False. If the bucket is missing, the script proceeds to create it using the create_bucket method, ensuring the data source bucket is always available.

In [17]:
%%time
# Check if s3 bucket exists, and if not create S3 bucket for knowledge base data source

# Try-except block to handle potential errors
try:

    # Define the bucket name (you can replace this with your actual bucket name)
    bucket_name = s3_bucket_name

    # Check if the bucket exists
    def check_bucket_exists(bucket_name):
        try:
            s3_client.head_bucket(Bucket=bucket_name)
            print(f"Bucket '{bucket_name}' already exists.")
            return True
        except botocore.exceptions.ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == '404':
                print(f"Bucket '{bucket_name}' does not exist.")
                return False
            else:
                raise e

    # If the bucket doesn't exist, create it
    if not check_bucket_exists(bucket_name):
        # Create the S3 bucket
        s3_client.create_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' created successfully.")

except botocore.exceptions.BotoCoreError as boto_error:
    print(f"An error occurred with Boto3: {boto_error}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

Bucket 'bedrock-kb-us-east-1-027437746815-565' does not exist.
Bucket 'bedrock-kb-us-east-1-027437746815-565' created successfully.
CPU times: user 24.9 ms, sys: 1.35 ms, total: 26.3 ms
Wall time: 198 ms


# Download and prepare dataset

In [18]:
%%time
# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current working directory: {current_directory}")

# Construct the path to 'data/rag_use_cases' inside the current directory
data_directory = os.path.join(current_directory, 'data', 'pretraining')

# Print the resulting path
print(f"Data directory path: {data_directory}")

Current working directory: /root/chapter9
Data directory path: /root/chapter9/data/pretraining
CPU times: user 1.08 ms, sys: 246 μs, total: 1.33 ms
Wall time: 1.47 ms


# Understand dataset 

### Source of the datasets: Amazon-com-Inc-2023-Annual-Report.pdf

In [24]:
%%time

# Define the output file path
train_file_name = "amazon-annual-report-dataset.jsonl"
train_dataset_filename = f"{data_directory}/{train_file_name}"
sample_file_name = "Amazon-com-Inc-2023-Annual-Report.pdf"  
sample_dataset_filename = f"{data_directory}/{sample_file_name}"

# Function to split PDF text and create the dataset file
def create_dataset(file_name, dataset_folder="data", chunk_size=20000, chunk_overlap=2000):
    try:
        # Load the PDF document
        print(f"Loading PDF file: {file_name}")
        loader = PyPDFLoader(file_name)
        document = loader.load()
        print(f"Loaded {len(document)} pages from the document.")

        # Split the document into chunks using RecursiveCharacterTextSplitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,  # Set chunk size
            chunk_overlap=chunk_overlap  # Set overlap for continuity across chunks
        )

        # Split the document into smaller text chunks
        docs = text_splitter.split_documents(document)
        print(f"Document split into {len(docs)} chunks.")

        # Prepare dataset content in the required format {"input": "<input text>"}
        contents = ""
        for doc in docs:
            content = {"input": doc.page_content}
            contents += (json.dumps(content) + "\n")

        # Ensure the dataset folder exists, create if necessary
        if not os.path.exists(dataset_folder):
            print(f"Dataset folder '{dataset_folder}' not found. Creating it.")
            os.makedirs(dataset_folder)


        # Write the dataset content to a file
        with open(train_dataset_filename, "w") as file:
            file.writelines(contents)
            print(f"Dataset saved successfully to {train_dataset_filename}")

    except FileNotFoundError as e:
        # Handle file not found error
        print(f"Error: The file {file_name} was not found. Please check the file path. {e}")
    except json.JSONDecodeError as e:
        # Handle JSON decoding errors
        print(f"Error: Failed to decode JSON while processing the document. {e}")
    except Exception as e:
        # Catch any other unexpected errors
        print(f"Unexpected error occurred while creating the dataset: {e}")

# Example usage
create_dataset( sample_dataset_filename, data_directory )

Loading PDF file: /root/chapter9/data/pretraining/Amazon-com-Inc-2023-Annual-Report.pdf
Loaded 92 pages from the document.
Document split into 91 chunks.
Dataset saved successfully to /root/chapter9/data/pretraining/amazon-annual-report-dataset.jsonl
CPU times: user 4.45 s, sys: 82 ms, total: 4.53 s
Wall time: 6.35 s


# Disclaimer
##### Make Sure that data_directory is pointing to the right path and data files are present. Otherwise, you need to change the above code

### The code defines a function upload_to_s3 that uploads files to an S3 bucket with error handling for common issues such as missing files or upload failures. The function first checks if the file exists locally. If the file is found, it attempts to upload the file to the S3 bucket using s3_client.upload_file.  After uploading, the function returns the S3 URI for the uploaded file. If any file upload fails, the function returns None. The S3 URIs for the uploaded files (training, validation, and test datasets) are printed if all uploads are successful.

In [25]:
%%time

# Function to upload files to S3 with error handling
def upload_to_s3(file_name, local_path, s3_bucket, s3_key):
    try:
        # Check if the file exists locally
        if not os.path.exists(local_path):
            raise FileNotFoundError(f"File {local_path} not found.")
        
        try:
            # Attempt to upload file to S3
            s3_client.upload_file(local_path, s3_bucket, s3_key)
            print(f"Successfully uploaded {file_name} to s3://{s3_bucket}/{s3_key}")
            return f"s3://{s3_bucket}/{s3_key}"
        except boto3.exceptions.S3UploadFailedError as e:
            print(f"S3 upload failed for {file_name}: {e}")
        except boto3.exceptions.NoCredentialsError as e:
            print(f"No AWS credentials found for {file_name}: {e}")
        except Exception as e:
            print(f"Unexpected error while uploading {file_name}: {e}")

    except FileNotFoundError as e:
        print(f"File not found error for {file_name}: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    return None  # Return None if upload failed


# Upload the files and store their S3 URIs in the respective variables
train_file_path_s3_uri = upload_to_s3(train_file_name, os.path.join(data_directory, train_file_name), s3_bucket_name, train_file_name)

# If all files are successfully uploaded, print the S3 URIs
if train_file_path_s3_uri:
    print("S3 URIs for the datasets:")
    print(f"Train URI: {train_file_path_s3_uri}")
else:
    print("One or more files failed to upload.")

Successfully uploaded amazon-annual-report-dataset.jsonl to s3://bedrock-kb-us-east-1-027437746815-565/amazon-annual-report-dataset.jsonl
S3 URIs for the datasets:
Train URI: s3://bedrock-kb-us-east-1-027437746815-565/amazon-annual-report-dataset.jsonl
CPU times: user 20.1 ms, sys: 6.2 ms, total: 26.3 ms
Wall time: 168 ms


# Fine out role ARN

In [26]:
%%time

# Find out IAM role and ARN for this session

def find_iam_role_by_name_substring(substring):
    try:
        # Use list_roles to retrieve IAM roles
        response = iam_client.list_roles()

        # Filter roles by name that contains the substring
        matching_roles = [role for role in response['Roles'] if substring in role['RoleName']]

        if matching_roles:
            for role in matching_roles:
                print(f"Found Role: {role['RoleName']} | ARN: {role['Arn']}")
                genaibookedbedrocksagemakerexecutionrolearn = role['Arn']
        else:
            print(f"No roles found with name containing '{substring}'.")
            
        return genaibookedbedrocksagemakerexecutionrolearn

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Call the function with the desired substring
genaibookedbedrocksagemakerexecutionrolearn = find_iam_role_by_name_substring("GenAIBookBedrockSageMakerExecutionRole")

Found Role: book-demo-GenAIBookBedrockSageMakerExecutionRole-ppFsemKcIlvD | ARN: arn:aws:iam::027437746815:role/book-demo-GenAIBookBedrockSageMakerExecutionRole-ppFsemKcIlvD
CPU times: user 23.5 ms, sys: 3.73 ms, total: 27.2 ms
Wall time: 100 ms


# Define variable for pretraining

In [27]:
bedrock_llm_foundation_model = "amazon.titan-text-lite-v1:0:4k"


customization_job_name_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
bedrock_model_customization_job_name = f"bedrock-model-pretrain-job-{customization_job_name_suffix}"
bedrock_model_customization_model_name = f"bedrock-pretrain-model-{customization_job_name_suffix}"
# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING". 
bedrock_model_customization_type = "CONTINUED_PRE_TRAINING"
bedrock_model_provisioned_model_name = f"bedrock-provisioned-model-{customization_job_name_suffix}"

### Amazon Titan text model customization hyperparameters:

    epochs: The number of complete passes through the training dataset. This parameter can take any integer value between 1 and 10, with a default value of 5.
    batchSize: The number of samples processed before updating the model's parameters. It can take any integer value between 1 and 64, with a default value of 1.
    learningRate: The rate at which the model's parameters are updated after each batch. This parameter can be any float value between 0.0 and 1.0, with a default value of 1.00E-5.
    learningRateWarmupSteps: The number of iterations during which the learning rate is gradually increased to the specified value. This parameter can take any integer value between 0 and 250, with a default value of 5.

##### Refer: https://docs.aws.amazon.com/bedrock/latest/userguide/cm-hp-titan-text.html

#### The code defines the hyperparameters for pretraining the Titan text model, including the number of epochs, batch size, and learning rate. It then specifies the S3 URIs for the training, validation (optional), and output data. The code constructs the necessary configurations for the training and validation datasets and the output location. It attempts to create a model customization job using AWS Bedrock's create_model_customization_job function, providing the required parameters such as the customization type, job name, custom model name, and role ARN. 

In [28]:

%%time

# Define the hyperparameters for pretraining Titan text model
hyper_parameters = {
    "epochCount": "1",
    "batchSize": "1",
    "learningRate": "0.00003",
}

output_file_path_s3_uri = f's3://{s3_bucket_name}/outputs/output-{bedrock_model_customization_model_name}'

# Specify your data path for training, validation (optional), and output
training_data_config = {"s3Uri": train_file_path_s3_uri}


output_data_config = {"s3Uri": output_file_path_s3_uri}

# Try to create the customization job
try:
    # Create the customization job
    training_job_response = boto3_bedrock_client.create_model_customization_job(
        customizationType=bedrock_model_customization_type,
        jobName=bedrock_model_customization_job_name,
        customModelName=bedrock_model_customization_model_name,
        roleArn=genaibookedbedrocksagemakerexecutionrolearn,
        baseModelIdentifier=bedrock_llm_foundation_model,
        hyperParameters=hyper_parameters,
        trainingDataConfig=training_data_config,
        outputDataConfig=output_data_config
    )
    
    print("Customization job created successfully.")
    print(training_job_response)

except KeyError as e:
    print(f"Missing required parameter: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

Customization job created successfully.
{'ResponseMetadata': {'RequestId': '3bf8ff15-42e3-46dc-85b9-e15c84c00188', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Fri, 06 Dec 2024 14:40:15 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '3bf8ff15-42e3-46dc-85b9-e15c84c00188'}, 'RetryAttempts': 0}, 'jobArn': 'arn:aws:bedrock:us-east-1:027437746815:model-customization-job/amazon.titan-text-lite-v1:0:4k/mdmwnk0gvrbk'}
CPU times: user 14.7 ms, sys: 3.52 ms, total: 18.2 ms
Wall time: 236 ms


#### The code checks the status of a pretraining job in a loop until it is no longer "InProgress." It defines a function check_pretraining_job_status() to fetch the job status. Initially, the job status is checked, and if it is "InProgress," the status is checked every 60 seconds. Once the job status is no longer "InProgress," the job details are retrieved, and the output job name is derived from the job ARN.

In [ ]:

%%time

# Function to check the status of the pretraining job
def check_pretraining_job_status(job_name):
    try:
        # Fetch the current status of the pretraining job
        job_status = boto3_bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]
        return job_status
    except KeyError as e:
        print(f"Error: Missing expected data in the response: {e}")
    except boto3.exceptions.Boto3Error as e:
        print(f"Boto3 error occurred: {e}")
    except Exception as e:
        print(f"Unexpected error occurred while fetching job status: {e}")
    return None

# Check the initial job status
pretraining_job = check_pretraining_job_status(bedrock_model_customization_job_name)
if pretraining_job:
    print(f"Initial pretraining job status: {pretraining_job}")

    # Loop to check the status every 60 seconds until the job is no longer "InProgress"
    while pretraining_job == "InProgress":
        print("Job is still in progress, checking again after 60 seconds...")
        time.sleep(60)  # Wait for 60 seconds before checking the status again
        pretraining_job = check_pretraining_job_status(bedrock_model_customization_job_name)
        if pretraining_job:
            print(f"Current pretraining job status: {pretraining_job}")
    
    # Once the job is no longer "InProgress", fetch and display the final job details
    if pretraining_job != "InProgress":
        pretraining_job_details = boto3_bedrock_client.get_model_customization_job(jobIdentifier=bedrock_model_customization_job_name)
        print(pretraining_job_details)  
        output_job_name = "model-customization-job-" + pretraining_job_details['jobArn'].split('/')[-1]
        print(f"Output job name: {output_job_name}")
else:
    print("Error: Could not retrieve the pretraining job status.")


Initial pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progress, checking again after 60 seconds...
Current pretraining job status: InProgress
Job is still in progr

# Test not done this point onwards

# Test not done this point onwards

# Test not done this point onwards

# Test not done this point onwards

#### Provisioned throughput is required not only for evaluating the model's performance but also for handling custom model inferences. You specify provisioned throughput in Model Units (MU), where each model unit defines the throughput capacity for a given model. The MU determines the number of input and output tokens the model can process and generate per minute.

#### For custom models, provisioned throughput ensures that the model can handle inference requests efficiently, especially when dealing with large volumes of data or high-frequency requests. Without sufficient throughput, the model may experience delays or be unable to process requests within an acceptable timeframe.

#### Model unit quotas depend on the level of commitment to provisioned throughput. For custom models with no commitment, you are allocated one model unit per throughput, with a limit of two provisioned throughputs per account. For custom models with commitment, the default quota is 0 model units, and you may request an increase if necessary.

#### Provisioned throughput is also required after a customization job is finished, to ensure the pretraining model can be used effectively for inference. You can create provisioned throughput either through the AWS console or using the relevant API call. It typically takes around 20-30 minutes to complete the provisioning process.

In [ ]:
%%time

# Retrieve the custom model ARN (model identifier)
try:
    custom_model_id_arn = boto3_bedrock_client.get_custom_model(modelIdentifier=bedrock_model_customization_model_name)['modelArn']
    print(f"Custom model ARN: {custom_model_id_arn}")
except KeyError as e:
    print(f"Error: Custom model with identifier {bedrock_model_customization_model_name} not found: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred while fetching model ARN: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

# Create Provisioned Throughput
try:
    # Create provisioned throughput for the model
    provisioned_model_response = boto3_bedrock_client.create_provisioned_model_throughput(
        modelUnits=1,
        provisionedModelName=bedrock_model_provisioned_model_name,
        modelId=bedrock_llm_foundation_model
    )
    provisioned_model_id_arn = provisioned_model_response['provisionedModelArn']
    print(f"Provisioned throughput ARN: {provisioned_model_id_arn}")
    
except KeyError as e:
    print(f"Error: Failed to create provisioned throughput: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred while creating provisioned throughput: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

# Check the status of the provisioned throughput until it's completed
try:
    status_provisioning = boto3_bedrock_client.get_provisioned_model_throughput(provisionedModelId=provisioned_model_id_arn)['status']
    print(f"Provisioned throughput status: {status_provisioning}")

    while status_provisioning == 'Creating':
        time.sleep(60)  # Wait for a minute before checking the status again
        status_provisioning = boto3_bedrock_client.get_provisioned_model_throughput(provisionedModelId=provisioned_model_id_arn)['status']
        print(f"Provisioned throughput status: {status_provisioning}")
        time.sleep(60)  # Wait for another minute before the next status check

except KeyError as e:
    print(f"Error: Failed to retrieve provisioning status: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred while checking provisioning status: {e}")
except Exception as e:
    print(f"Unexpected error occurred while checking provisioning status: {e}")


#### The provided code demonstrates how to invoke both a pretraining model and a base model using the AWS Bedrock service. First, it loads test data, extracting the prompt and reference summary for the model evaluation. The prompt is used to create the request body for invoking the models. The base model's ARN is specified, and the body includes parameters such as maxTokenCount, stopSequences, and others to control the generation. It then uses the boto3 client to invoke both the pretraining model and the base model. The responses are parsed, and the results from both models, along with the reference summary, are printed. 

In [ ]:
%%time

# Invoke the Custom Model
try:
    # Load test data and extract prompt and reference summary
    with open(test_file_path) as f:
        lines = f.read().splitlines()
    
    # Get the prompt and reference summary from the test data
    test_prompt = json.loads(lines[10])['prompt']
    reference_summary = json.loads(lines[3])['completion']
    print("Test prompt: ", test_prompt)
    print("\n\n")
    print("Reference summary: ", reference_summary)
    print("\n\n")

    # Prepare the prompt for model invocation
    prompt = f"{test_prompt}"

    # Define the base model ARN
    bedrock_llm_foundation_model_arn = f'arn:aws:bedrock:{aws_region_name}::foundation-model/{bedrock_llm_foundation_model}'

    # Create the body for the model invocation request
    body = json.dumps({
        "inputText": prompt,
        "textGenerationConfig": {
            "maxTokenCount": 2048,
            "stopSequences": ['User:'],
            "temperature": 0,
            "topP": 0.9
        }
    })

    # Set request headers
    accept = 'application/json'
    contentType = 'application/json'

    # Invoke the pretraining model
    pretraining_response = boto3_bedrock_runtime_client.invoke_model(
        body=body, 
        modelId=provisioned_model_id_arn, 
        accept=accept, 
        contentType=contentType
    )

    # Invoke the base model
    base_model_response = boto3_bedrock_runtime_client.invoke_model(
        body=body, 
        modelId=bedrock_llm_foundation_model_arn, 
        accept=accept, 
        contentType=contentType
    )

    # Parse the responses from both models
    pretraining_response_body = json.loads(pretraining_response.get('body').read())
    base_model_response_body = json.loads(base_model_response.get('body').read())

    # Print the responses from the models
    print("Base model response: ", base_model_response_body["results"][0]["outputText"] + '\n')
    print("\n\n")
    print("pretraining model response:", pretraining_response_body["results"][0]["outputText"] + '\n')
    print("\n\n")
    print("Reference summary from test data: ", reference_summary)
    print("\n\n")

except FileNotFoundError as e:
    print(f"Error: Test file not found at {test_file_path}: {e}")
except json.JSONDecodeError as e:
    print(f"Error: Failed to decode JSON from the test file: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred while invoking the model: {e}")
except KeyError as e:
    print(f"Error: Missing key in the model response: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")'''

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html


#### <ins>Step 2</ins> 

#### If you are not executing any further lab of this Chapter 10
##### Uncomment and execute the below code to delete the provision thoughtput and custom model

In [ ]:
'''
%%time 

# Function to delete provisioned throughput
def delete_provisioned_throughput(provisioned_model_id):
    try:
        # Attempt to delete provisioned model throughput
        print(f"Attempting to delete provisioned throughput for model ID: {provisioned_model_id}")
        response = boto3_bedrock_client.delete_provisioned_model_throughput(provisionedModelId=provisioned_model_id)
        
        # Log the response if successful
        print("Provisioned throughput deletion successful. Response:")
        print(response)
        
    except ClientError as e:
        # Handle client error (e.g., invalid request or resource not found)
        print(f"Client error occurred while deleting provisioned throughput: {e}")
    except Exception as e:
        # Handle any other unexpected errors
        print(f"Unexpected error occurred while deleting provisioned throughput: {e}")

# Function to delete a custom model
def delete_custom_model(model_identifier):
    try:
        # Attempt to delete the custom model
        print(f"Attempting to delete custom model with identifier: {model_identifier}")
        response = boto3_bedrock_client.delete_custom_model(modelIdentifier=model_identifier)
        
        # Log the response if successful
        print("Custom model deletion successful. Response:")
        print(response)
        
    except ClientError as e:
        # Handle client error (e.g., invalid request or resource not found)
        print(f"Client error occurred while deleting custom model: {e}")
    except Exception as e:
        # Handle any other unexpected errors
        print(f"Unexpected error occurred while deleting custom model: {e}")

# Delete provisioned throughput for the custom model
delete_provisioned_throughput(provisioned_model_id_arn)

# Delete the custom model
delete_custom_model(bedrock_model_customization_model_name)
'''